In [2]:
# This is work in progres 
# I used the encodere from the : Quantum autoencoders with enhanced data encoding
# paper link:https://arxiv.org/abs/2010.06599

# it is working but data and optimization parameters need to be modified for an exact comparison.
# Also, as I observed, they used a scalar as an extra feature. I think we can use a vector instead, and maybe we will get better results. 

### Pip install

In [3]:
#!pip install pennylane --upgrade

## Imports

In [42]:
import random

import pennylane as qml
from pennylane import numpy as np

from pennylane.optimize import AdamOptimizer

import torch
from torchvision import datasets, transforms

In [43]:
from qencode.initialize import setAB_amplitude, setAux, setEnt
from qencode.encoders import e3_enhance
from qencode.training_circuits import swap_t
from qencode.qubits_arrangement import QubitsArrangement

## Get Data

In [44]:
def get_dataset(img_width, img_height, train):
    trainset = datasets.MNIST(root='./dataset', train=train, download=True,
                              transform=transforms.Compose(
                                  [transforms.Resize((img_width, img_height)), transforms.ToTensor()])
                              )
    return trainset

In [45]:
nr_qubits = 6
input_data = get_dataset(img_width=8, img_height=8, train=True)
input_data[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0039, 0.1137, 0.2510, 0.3647, 0.3529, 0.2784, 0.0118],
          [0.0000, 0.0157, 0.4078, 0.7725, 0.5608, 0.2588, 0.0902, 0.0039],
          [0.0000, 0.0000, 0.0353, 0.4157, 0.2745, 0.0353, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0824, 0.4941, 0.3882, 0.0118, 0.0000],
          [0.0000, 0.0039, 0.0510, 0.3059, 0.6392, 0.4784, 0.0157, 0.0000],
          [0.0078, 0.2392, 0.5804, 0.6118, 0.2667, 0.0392, 0.0000, 0.0000],
          [0.0078, 0.1451, 0.1843, 0.0588, 0.0000, 0.0000, 0.0000, 0.0000]]]),
 5)

In [46]:
len(input_data)

60000

In [47]:
filtered_data = [image for image in input_data if image[1] in [0, 1]]
input_data = filtered_data
len(filtered_data)

12665

## Training node


In [63]:
shots = 2500
spec = QubitsArrangement(nr_trash=3, nr_latent=3, nr_swap=1, nr_ent=0)
print("Qubits:", spec.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=spec.num_qubits)

nr_layers = 4

Qubits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [49]:
@qml.qnode(dev)
def training_circuit_example(init_params, encoder_params, reinit_state, x):
    # Initialization
    setAB_amplitude(spec, init_params)
    setAux(spec, reinit_state)
    setEnt(spec, inputs=[1 / np.sqrt(2), 0, 0, 1 / np.sqrt(2)])

    #encoder 
    for params in encoder_params:
        e3_enhance(params, x, spec)

    #swap test 
    swap_t(spec)

    return [qml.probs(i) for i in spec.swap_qubits]


## Training parameters 

In [50]:
epochs = 50
learning_rate = 0.0003
batch_size = 5
num_samples = 20

beta1 = 0.9
beta2 = 0.999
opt = AdamOptimizer(learning_rate, beta1=beta1, beta2=beta2)

In [51]:
def fid_func(output):
    # Implemented as the Fidelity Loss
    # output[0] because we take the probability that the state after the 
    # SWAP test is ket(0), like the reference state
    fidelity_loss = 1 / output[0]
    return fidelity_loss

In [60]:
def cost(encoder_params, X):
    reinit_state = [0 for _ in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0][0],
                                          encoder_params=encoder_params,
                                          reinit_state=reinit_state, x=x[1])[0]
        f = fid_func(output)
        loss = loss + f
    return loss / len(X)

In [53]:
def fidelity(encoder_params, X):
    reinit_state = [0 for _ in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0][0],
                                          encoder_params=encoder_params,
                                          reinit_state=reinit_state, x=x[1])[0]
        f = output[0]
        loss = loss + f
    return loss / len(X)

In [54]:
def iterate_batches(X, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [55]:
training_data = [input_data[i] for i in range(num_samples)]

In [56]:
batches = iterate_batches(X=training_data, batch_size=batch_size)
batches[0]

[[tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0471, 0.3804, 0.0941, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0039, 0.3294, 0.5725, 0.0627, 0.0000, 0.0000, 0.0000, 0.0000,
           0.1255, 0.6275, 0.1843, 0.0000, 0.0000, 0.0000, 0.0000, 0.0314, 0.5490,
           0.4157, 0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.2314, 0.6902, 0.1020,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.4157, 0.4353, 0.0078, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0980, 0.0902, 0.0000, 0.0000, 0.0000,
           0.0000]]),
  0],
 [tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0471, 0.3804, 0.0941, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0039, 0.3294, 0.5725, 0.0627, 0.0000, 0.0000, 0.0000, 0.0000,
           0.1255, 0.6275, 0.1843, 0.0000, 0.0000, 0.0000, 0.0000, 0.0314, 0.5490,
           0.4157, 0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 

In [57]:
X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in training_data]
X2 = []
for i in range(len(X1)):
    X2.append([X1[1], training_data[i][1]])
X = X2

#for x in X:
#    print(x[0],x[1])

In [65]:
# initialize random encoder parameters
nr_encod_qubits = len(spec.trash_qubits) + len(spec.latent_qubits)
nr_par_encoder = nr_layers * 2 * nr_encod_qubits + 2 * len(spec.trash_qubits)
print(nr_par_encoder)
encoder_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)

for epoch in range(epochs):
    batches = iterate_batches(X=training_data, batch_size=batch_size)
    for xbatch in batches:
        encoder_params = opt.step(cost, encoder_params, X=xbatch)

    loss_training = cost(encoder_params, X)
    fidel = fidelity(encoder_params, X)

    print("Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_training, fidel))

54
Epoch:0 | Loss:1.8787083842368628 | Fidelity:0.532562782076927


KeyboardInterrupt: 